In [1]:
import pandas as pd
import numpy as np

In [2]:
data_pddf = pd.read_excel('./input_files/positions.xlsx')

In [3]:
data_pddf.columns

Index([           u'Member',                u'CC',             u'Delta',
               u'UDL value',        u'Unnamed: 4',        u'Unnamed: 5',
                     u'UDL',                u'Nu',              u'Coef',
              u'Unnamed: 9', u'99,70% shift rel+', u'99,70% shift abs+',
               u'UPSR PROD',  u'UPSR Is Relative'],
      dtype='object')

In [4]:
positions_pddf = data_pddf[['Member', 'CC', 'Delta']]
positions_pddf.head()

,Member,CC,Delta
0,PB1,FCE,-150.0
1,PB2,AEX,600.0
2,PB2,FCE,-90.0
3,PB3,AEX,-1200.0
4,PB3,FCE,390.0


In [5]:
udl_values_df = data_pddf[['CC', 'UDL value']]
udl_values_df = udl_values_df.drop_duplicates('CC').set_index('CC').sort_index()
udl_values_df.head()

,UDL value
CC,
AAI,29.66
AEX,443.83
AF,8.62
AGN,5.08
AH,20.60


In [6]:
udl_pddf = data_pddf[['UDL', 'Nu', 'Coef']]
udl_pddf = udl_pddf.set_index('UDL').sort_index()
udl_pddf = udl_pddf.dropna()
udl_pddf.head()

,Nu,Coef
UDL,,
AAI,3.133782,0.020107
AEX,3.345641,0.013696
AF,4.551459,0.029619
AGN,2.656825,0.023799
AH,2.705614,0.015807


In [7]:
udl_pddf['UDL value'] = udl_values_df['UDL value']
udl_pddf.head()

,Nu,Coef,UDL value
UDL,,,
AAI,3.133782,0.020107,29.66
AEX,3.345641,0.013696,443.83
AF,4.551459,0.029619,8.62
AGN,2.656825,0.023799,5.08
AH,2.705614,0.015807,20.60


In [8]:
members_id = list(set(positions_pddf['Member'].values))
members_id.sort(key=lambda s: int(s[2:]))

assets_id = list(set(udl_values_df.index.values))
assets_id.sort()

In [9]:
df_positions_matrix = pd.DataFrame(index=assets_id, columns=members_id)

for member in members_id:
    tmp_pos = positions_pddf.loc[positions_pddf['Member'] == member]
    tmp_pos = tmp_pos[['CC', 'Delta']].set_index('CC')['Delta']
    
    df_positions_matrix[member] = tmp_pos
    
df_positions_matrix[np.isnan(df_positions_matrix)] = 0.

df_positions_matrix = df_positions_matrix.transpose()

df_positions_matrix.head()

,AAI,AEX,AF,AGN,AH,AHA,AI,AKZ,AL1,ASL,...,UCB,UG,UL,UMC,UN,VIE,VK,VPK,WHV,WKL
PB1,0.00,0.000,0.00,0.00,0.0,0.0,0.0000,0.00,0.0,0.0000,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.0
PB2,0.00,600.000,0.00,0.00,0.0,0.0,0.0000,0.00,0.0,0.0000,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.0
PB3,0.00,-1200.000,0.00,0.00,0.0,0.0,0.0000,0.00,0.0,0.0000,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.0
PB4,0.00,116200.000,0.00,0.00,0.0,0.0,-200.2320,0.00,0.0,0.0000,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.00,0.00,0.0
PB5,2913.25,200149.546,62689.56,1251773.09,220357.6,0.0,521.7448,34769.11,0.0,51224.8981,...,4651.92,8907.06,2564.61,-8852.76,157406.06,-174.95,23508.6628,3601.46,50772.03,-175655.6


In [10]:
import os

dir_ = './transformed_input'

if not os.path.exists(dir_):
    os.makedirs(dir_)

In [11]:
udl_pddf.to_csv(os.path.join(dir_, 'UDL.csv'))
df_positions_matrix.to_csv(os.path.join(dir_, 'positions.csv'))